# Tệp nhật ký - Trực quan hóa dữ liệu

### Thư viện được sử dụng
##### Spark Session, Dataframe Functions, Pandas, Dash, Dash core components, html components, Input, Output and state dependencies, Plotly Graph objects, time and date time

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
from datetime import datetime,timedelta
import time

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  


### Spark Session
##### Tạo đối tượng Spark Session với dữ liệu cấu hình stax trình kết nối spark-cassandra và thông tin xác thực kết nối liên quan đến cassandra.

In [2]:
spark = SparkSession.builder.appName("pyspark-notebook").\
config("spark.jars.packages","com.datastax.spark:spark-cassandra-connector_2.12:3.0.0,com.datastax.spark:spark-cassandra-connector-driver_2.12:3.0.0").\
config("spark.cassandra.connection.host","cassandra").\
config("spark.cassandra.auth.username","cassandra").\
config("spark.cassandra.auth.password","cassandra").\
getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
com.datastax.spark#spark-cassandra-connector-driver_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3a9d231c-fc1d-404a-8294-489a8d0da44d;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.0.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.0.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.7.2 in central
	found com.datastax.oss#native-protocol;1.4.10 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.3.4 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics

### Truy xuất dữ liệu từ Cassandra
##### Một phương thức chung để đọc dữ liệu từ cassandra. Đưa ra một điều kiện để lọc dữ liệu từ khung dữ liệu, tên trường để tổng hợp và tham số có giới hạn một số dữ liệu hay không. Trả về khung dữ liệu Pandas

In [3]:
def read_cassandra(filter_condition,group_by,limit=False):
    logs_df = spark\
             .read\
             .format("org.apache.spark.sql.cassandra")\
             .options(table="nasalog", keyspace="loganalysis")\
             .load()\
             .filter(filter_condition)
    agg_df =logs_df.groupBy(group_by).count().sort(group_by)
    if limit:
        return agg_df.limit(5).toPandas()
    else:
        return agg_df.toPandas()

### Truy xuất dữ liệu từ HDFS
#####  Một phương pháp chung để đọc dữ liệu từ HDFS. Lấy tên trường để tổng hợp dữ liệu, định dạng thời gian tùy chọn trong chuỗi và điều kiện bộ lọc boolean tùy chọn. Trả về khung dữ liệu Pandas.

In [4]:
schema="host string,time string,method string,url string,response string,bytes string"
def unique_hosts(group_by,time_format=None,filter_resp=False):
    logs_df = spark\
             .read\
             .csv("hdfs://namenode:8020/output/nasa_logs/",schema=schema)
    if time_format:
        logs_df = logs_df.withColumn(group_by,date_format(from_unixtime(col("time")),time_format))#.orderBy("time")
    if filter_resp:
        logs_df = logs_df.filter("response==404")
    agg_df =logs_df.limit(80000).groupBy(group_by).count().sort(group_by)
    return agg_df.toPandas()

### Tạo ứng dụng Dash
##### Tạo đối tượng ứng dụng Dash Multipage và định nghĩa bố cục ứng dụng.

In [5]:
app = dash.Dash(__name__, suppress_callback_exceptions=True)

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
], style={'textAlign': 'center'})

### Tạo tiêu đề và bảng
##### Định nghĩa về kiểu tiêu đề và tạo bảng có hai cột. Một hàng tiêu đề và các hàng dữ liệu.

In [6]:
#Color assignment
colors = {
    'background': 'white',#'#0C0F0A',
    'text': '#FFFFFF'
}

def create_header(title):
    header_style = {
        'background-color' : '#1B95E0',
        'padding' : '1.5rem',
        'color': 'white',
        'font-family': 'Verdana, Geneva, sans-serif'
    }
    header = html.Header(html.H1(children=title, style=header_style))
    return header

def generate_table(df, max_rows=10):
    table = html.Table(className="responsive-table",
                      children=[
                          html.Thead(
                              html.Tr(
                                  children=[html.Th(col.title()) for col in df.columns.values]
                                  
                                  ),style={'border':'1px black solid'}
                              ),
                          html.Tbody(
                              [
                              html.Tr(
                                  children=[html.Td(data) for data in d]
                                  )
                               for d in df.values.tolist()],style={'border':'1px black solid'})
                          ]
                       , style={'marginLeft': 'auto', 'marginRight': 'auto'}
    )
    
    return table

### Tạo trang
##### Định nghĩa trang chỉ mục, thời gian thực, hàng giờ và hàng ngày.

In [7]:
index_page = html.Div([
    html.Div([create_header('Log Analysis - Dashboard')]),
    dcc.Link('Go to Realtime Dash Board', href='/real-time'),
    html.Br(),
    dcc.Link('Go to Hourly Dash Board', href='/hourly'),
    html.Br(),
    dcc.Link('Go to Daily Dash Board', href='/daily'),
])

realtime_dashboard = html.Div(style={'backgroundColor': colors['background']}, children=
    [   
        html.Div([create_header('Log Analysis - Realtime Dashboard')]),
        html.Div([dcc.Graph(id='live-graph', animate=False)
                 ]
                 ,style={'width': '100%', 'display': 'inline-block'}
                ),
        html.Div([dcc.Graph(id='live-graph1', animate=False)
                 ]
                 ,style={'width': '100%', 'display': 'inline-block'}
                ),
        html.Div([dcc.Graph(id='live-graph2', animate=False)
                 ]
                 ,style={'width': '100%', 'display': 'inline-block'}
                ),
        html.Div([html.H2("Top Paths"), 
                  html.Div(id="top-paths-table")]
                 ,style={'width': '50%', 'display': 'inline-block', 'border':'2px black solid'}
                ),
        ##Intervals define the frequency in which the html element should be updated
        dcc.Interval(id='graph-update',interval=60*1000, n_intervals=0),
        html.Div(id='real-time-content'),
        html.Br(),
        dcc.Link('Go to Hourly Dash Board', href='/hourly'),
        html.Br(),
        dcc.Link('Go to Daily Dash Board', href='/daily'),
        html.Br(),
        dcc.Link('Go back to home', href='/')
    ]
)

hourly_dashboard = html.Div(style={'backgroundColor': colors['background']}, children=
    [   
        html.Div([create_header('Log Analysis - Hourly Dashboard')]),
        html.Div([dcc.Graph(id='hourly-graph', animate=False)
                 ]
                 ,style={'width': '100%', 'display': 'inline-block'}
                ),
        ##Intervals define the frequency in which the html element should be updated
        dcc.Interval(id='hourly-graph-update',interval=60*1000, n_intervals=0),
        html.Div(id='hourly-content'),
        html.Br(),
        dcc.Link('Go to Daily Dash Board', href='/daily'),
        html.Br(),
        dcc.Link('Go to RealTime Dash Board', href='/real-time'),
        html.Br(),
        dcc.Link('Go back to home', href='/')
    ]
)

daily_dashboard = html.Div(style={'backgroundColor': colors['background']}, children=
    [   
        html.Div([create_header('Log Analysis - Daily Dashboard')]),
        html.Div([dcc.Graph(id='daily-graph', animate=False)
                 ]
                 ,style={'width': '100%', 'display': 'inline-block'}
                ),
        html.Div([dcc.Graph(id='daily-graph1', animate=False)
                 ]
                 ,style={'width': '100%', 'display': 'inline-block'}
                ),
        ##Intervals define the frequency in which the html element should be updated
        dcc.Interval(id='daily-graph-update',interval=60*1000, n_intervals=0),
        html.Div(id='daily-content'),
        html.Br(),
        dcc.Link('Go to RealTime Dash Board', href='/real-time'),
        html.Br(),
        dcc.Link('Go to Hourly Dash Board', href='/hourly'),
        html.Br(),
        dcc.Link('Go back to home', href='/')
    ]
)

### Bảng điều khiển thời gian thực
##### Gọi lại để cập nhật bảng điều khiển theo thời gian thực. Đọc dữ liệu từ cơ sở dữ liệu cassandra và cập nhật dữ liệu mới nhất cứ sau 60 giây. Vẽ dữ liệu trong biểu đồ phân tán, biểu đồ hình tròn, biểu đồ thanh và bảng dữ liệu. đây là con đường nóng trong kiến ​​trúc Lambda.

In [8]:
#Call back for live graph
@app.callback(Output('live-graph', 'figure'),
              Input('graph-update', 'n_intervals')
             )
def update_graph_scatter(n_intervals):
    try:
        processed_time = 0
        filter_condition = "CAST(response AS DECIMAL) IS NOT NULL and time_added >'"+str(processed_time)+"'"
        group_by = 'response'
        df = read_cassandra(filter_condition,group_by)
        processed_time = time.time()-60
        df.dropna(inplace=True)
        
        #Define X and Y axis values        
        X = df["response"]
        Y = df['count']
        
        #Scatter graph definition
        data = go.Scatter(
                x=X,
                y=Y
                )

        return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[X.min(),X.max()],title='Status Codes'),
                                                    yaxis=dict(range=[Y.min(),Y.max()],title='Count'),
                                                    title='Status graphing'
                                                   )
               } 

    except Exception as e:
        #File to capture exceptions
        with open('errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')
            
@app.callback(Output('live-graph1', 'figure'),
              Input('graph-update', 'n_intervals')
             )
def update_graph_scatter1(n_intervals):
    try:
        processed_time = 0
        filter_condition = "CAST(response AS DECIMAL) IS NOT NULL and time_added >'"+str(processed_time)+"'"
        group_by = 'response'
        df = read_cassandra(filter_condition,group_by)
        processed_time = time.time()-60
        df.dropna(inplace=True)
        
        #Define X and Y axis values        
        X = df["response"]
        Y = df['count']
        
        #Scatter graph definition
        data = go.Pie(
                labels=X,
                values=Y
                )

        return {'data': [data]
               } 

    except Exception as e:
        #File to capture exceptions
        with open('errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')

@app.callback(Output('live-graph2', 'figure'),
              Input('graph-update', 'n_intervals')
             )
def update_graph_scatter2(n_intervals):
    try:
        processed_time = 0
        filter_condition = "time_added >'"+str(processed_time)+"'"
        group_by = 'extension'
        df = read_cassandra(filter_condition,group_by)
        processed_time = time.time()-60
        df.dropna(inplace=True)
        
        #Define X and Y axis values        
        X = df["extension"]
        Y = df['count']
        
        #Scatter graph definition
        data = go.Bar(
                x=X,
                y=Y,
            #width=5
                )

        return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[X.min(),X.max()],title='Paths'),
                                                    yaxis=dict(range=[Y.min(),Y.max()],title='Number of Hits'),
                                                    title='Visualizing Paths'
                                                   )
               } 

    except Exception as e:
        #File to capture exceptions
        with open('errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')

@app.callback(Output('top-paths-table', 'children'),
              Input('graph-update', 'n_intervals')
             )
def update_top_urls(n_intervals):
    try:
        processed_time = 0
        filter_condition = "time_added >'"+str(processed_time)+"'"
        group_by = 'url'
        df = read_cassandra(filter_condition,group_by,True)
        processed_time = time.time()-60

        df = df[['url','count']]

        return generate_table(df, max_rows=5)
    except Exception as e:
        #File to capture exceptions
        with open('table_errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')

# Update the index
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/real-time':
        return realtime_dashboard
    elif pathname == '/hourly':
        return hourly_dashboard
    elif pathname == '/daily':
        return daily_dashboard
    else:
        return index_page

###  Bảng điều khiển hàng giờ
##### Gọi lại để cập nhật bảng điều khiển mỗi giờ. Đọc dữ liệu từ HDFS và cập nhật dữ liệu mới nhất hàng giờ. Vẽ dữ liệu dưới dạng biểu đồ Scatter. Đây là một con đường lạnh lùng trong Kiến trúc Lambda.

In [9]:
@app.callback(Output('hourly-graph', 'figure'),
              Input('hourly-graph-update', 'n_intervals')
             )
def update_hourly_scatter(n_intervals):
    try:
        group_by = "hour"
        time_format = 'yy-MM-dd-HH'
        df = unique_hosts(group_by,time_format)
        df.dropna(inplace=True)
        
        #Define X and Y axis values        
        X = df["hour"]
        Y = df['count']
        
        #Scatter graph definition
        data = go.Scatter(
                x=X,
                y=Y,
            #width=5
                    )

        return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[X.min(),X.max()],title='Hour in a day (yy-MM-dd-HH)'),
                                                    yaxis=dict(range=[Y.min(),Y.max()],title='Number of Hits'),
                                                    title='Unique Requests Per Hour'
                                                   )
               } 

    except Exception as e:
        #File to capture exceptions
        with open('errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')

### Trang tổng quan hàng ngày
##### Gọi lại để cập nhật bảng điều khiển mỗi ngày. Đọc dữ liệu từ HDFS và cập nhật dữ liệu mới nhất hàng giờ. Vẽ dữ liệu dưới dạng biểu đồ phân tán và biểu đồ thanh. Đây là một con đường lạnh lùng khác trong Kiến trúc Lambda.

In [10]:
@app.callback(Output('daily-graph', 'figure'),
              Input('daily-graph-update', 'n_intervals')
             )
def update_daily_scatter(n_intervals):
    try:
        group_by = "day"
        time_format = 'yy-MM-dd'
        df = unique_hosts(group_by,time_format)
        df.dropna(inplace=True)
        
        #Define X and Y axis values        
        X = df["day"]
        Y = df['count']
        
        #Scatter graph definition
        data = go.Scatter(
                x=X,
                y=Y
                )

        return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[X.min(),X.max()],title='Day'),
                                                    yaxis=dict(range=[Y.min(),Y.max()],title='Count'),
                                                    title='Unique Request Per Day'
                                                   )
               } 

    except Exception as e:
        #File to capture exceptions
        with open('errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')
            
@app.callback(Output('daily-graph1', 'figure'),
              Input('daily-graph-update', 'n_intervals')
             )
def update_daily_pie(n_intervals):
    try:
        group_by = "response"
        df = unique_hosts(group_by)
        df.dropna(inplace=True)
        
        #Define X and Y axis values        
        X = df["response"]
        Y = df['count']
        
        #Scatter graph definition
        data = go.Pie(
                labels=X,
                values=Y
                )

        return {'data': [data]
               } 

    except Exception as e:
        #File to capture exceptions
        with open('errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')


### Trang Chính
##### Quá trình thực thi Ứng dụng Dash bắt đầu từ đây

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=False, port=8050, host='0.0.0.0')

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.18.0.5:8050/ (Press CTRL+C to quit)
172.18.0.1 - - [07/Dec/2024 09:40:30] "GET / HTTP/1.1" 200 -
172.18.0.1 - - [07/Dec/2024 09:40:30] "GET /_dash-layout HTTP/1.1" 200 -
172.18.0.1 - - [07/Dec/2024 09:40:30] "GET /_dash-dependencies HTTP/1.1" 200 -
172.18.0.1 - - [07/Dec/2024 09:40:30] "POST /_dash-update-component HTTP/1.1" 200 -
